# Setup


In [ ]:
%uv pip install ase matcalc[matgl]

# Demonstration


In this demonstration, we will identify a transition state using ASE to showcase the NEB method. We will also carry out a vibrational mode analysis to confirm the identity of the transition state.

In this example, we will be looking at the structure of bulk silicon with a vacancy in it and modeling the diffusion of an Si atom into the vacancy.

For additional details on running NEB calculations, refer to the [ASE documentation](https://ase-lib.org/ase/neb.html).


Here is the setup. We start with bulk silicon, which you can get from any crystal structure database, the Materials Project, or within ASE. We will make a 3x3x3 supercell.


In [ ]:
from ase.build import bulk

bulk_crystal = bulk("Al") * (3, 3, 3)

For good measure, let's make sure to relax the bulk crystal structure, including the unit cell and positions. We will be using the same machine learning potential we have used before since it generally works well. Again, the choice of calculator is not the focal point yet and can be swapped out without having to change the rest of the code.


In [ ]:
from matcalc import load_fp

potential_name = "TensorNet-MatPES-r2SCAN-v2025.1-PES"

In [ ]:
from ase.optimize import BFGS
from ase.filters import FrechetCellFilter

bulk_crystal.calc = load_fp(potential_name)
opt = BFGS(FrechetCellFilter(bulk_crystal))
opt.run(fmax=0.01)

From here, we are going to introduce a vacancy, which means one of the silicon atoms in the crystal structure will be removed. We will remove the 0th index. This is a defect.


In [ ]:
initial = bulk_crystal.copy()
del initial[0]

Now let's take a look at what we have made. If you compare the bulk crystal with the defective one, it becomes clear that there is a vacancy.


In [ ]:
from ase.visualize import view

view([bulk_crystal, initial])

We are going to model the diffusion of one of the Si atoms into the vacancy. So, our final state will move one of the atoms into the vacancy position, which we will do here.


In [ ]:
# the Atoms object is mutable, so we use `.copy()`
final = initial.copy()

# final state: a neighboring atom hops into the vacancy
final[0].position = bulk_crystal[0].position

Let's compare the initial and final state. There must be a transition state between these two configurations.


In [ ]:
view([initial, final])

Let's start by making sure the initial and final state are at local minima in the potential energy surface by carrying out a geometry optimization. We will keep the unit cell fixed here because we are considering diffusion into a single vacancy in a large semi-infinite crystal, which in reality would not alter the lattice of the material.


In [ ]:
for atoms in (initial, final):
    atoms.calc = load_fp(potential_name)
    opt = BFGS(atoms)
    opt.run(fmax=0.01)

Great. Now we need to construct images for our NEB. We do this by linearly interpolating between the initial and final state. We will make 5 intermediates images, plus the end points that we have already optimized.


In [ ]:
n_intermediate_images = 7
images = [initial] + [initial.copy() for _ in range(n_intermediate_images)] + [final]

In [ ]:
from ase.mep import NEB

neb = NEB(images, method="improvedtangent", climb=True)
neb.interpolate(method="linear")  # linearly interpolate

It's always good practice to view the images before running an NEB calculation. Let's do that and make sure it doesn't look horrible.


In [ ]:
view(images)

Looks reasonable enough. Let's make sure all the intermediate images have a calculator.


In [ ]:
for image in images[1:-1]:
    image.calc = load_fp(potential_name)

And now let's run our NEB. This can be done like any other optimization process, except we apply the `BFGS` optimizer to the `NEB` object rather than the `Atoms` object. The ASE documentation says `BFGS` does not work well for climbing image NEB calculations, but it seems to work fine here.


In [ ]:
opt = BFGS(neb)
opt.run(fmax=0.01)

Time to visualize our trajectory.


In [ ]:
view(images)

To make sure things look good, let's inspect the MEP identified by the NEB calculation. It should only have one local maximum.


In [ ]:
from ase.mep import NEBTools

fig = NEBTools(images).plot_band()

Looks good! Now as a final sanity check, let's run a vibrational frequency calculation to ensure there is only one imaginary mode.


First, we need to identify the transition state. That's the one with the highest energy.


In [ ]:
import numpy as np

energies = [img.get_potential_energy() for img in images]
ts_index = np.argmax(energies)
ts = images[ts_index]

Then we run a vibrational frequency calculation. In solids, it is more typical to run a phonon calculation, but we will get to that later in the course. For now, a standard vibrational frequency calculation is fine.


In [ ]:
from ase.vibrations import Vibrations

vib = Vibrations(ts)
vib.clean()  # start fresh
vib.run()  # run vibrational analysis

In [ ]:
vib.summary()  # print out summary

Great! There is only one notable imaginary mode. The other modes near 0 are simply numerical noise and can be ignored.


Sanity check! There should be 3N modes. Let's confirm.


In [ ]:
print(f"We have {len(atoms)} atoms, so 3N = {3 * len(atoms)} modes are expected.")
print(f"We have {len(vib.get_frequencies())} modes calculated.")

Let's visualize the imaginary mode and make sure it corresponds to the reaction pathway.


In [ ]:
vib.write_mode(0)  # writes to vib.0.traj

In [ ]:
!ase gui vib.0.traj # opens GUI

Looks good!


Some questions to consider exploring:

- How does changing the bulk structure (e.g. the element) alter the diffusion barrier?
- How does changing the unit cell size influence the predicted barrier? This is a parameter that should be converged. Too large and it will be too expensive, but too small and finite size effects may play a role.
- How does the number of NEB images influence the transition state search process?
